In [1]:
import quinn

import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
1,application_1606928535611_0002,pyspark,idle,Link,Link


SparkSession available as 'spark'.
Connected. Call `.close()` to terminate connection gracefully.

In [2]:
from hops import hdfs

hdfs.project_name()

'amlsim'

In [3]:
import hops
print (hops.__version__)


2.0.0.1

In [4]:
from pyspark.sql import functions as F

AML_accounts_csv = spark.read\
             .option("inferSchema", "true")\
             .option("header", "true")\
             .format("csv")\
             .load("hdfs:///Projects/{}/Resources/accounts.csv".format(hdfs.project_name()))

AML_accounts_csv = AML_accounts_csv.drop('first_name')\
                        .drop('last_name')\
                        .drop('street_addr')\
                        .drop('city')\
                        .drop('state')\
                        .drop('zip')\
                        .drop('gender')\
                        .drop('birth_date')\
                        .drop('ssn')\
                        .drop('lon')\
                        .drop('lat')


#add random gender
gender_list = list(map(lambda c: F.lit(c), ['M', 'W']))
AML_accounts_csv = AML_accounts_csv.withColumn('gender', quinn.array_choice(F.array(gender_list)))


#add age based on initial_deposit
ages_young = list(map(lambda c: F.lit(c), range(10,25)))
ages_adult = list(map(lambda c: F.lit(c), range(25,100)))
AML_accounts_csv = AML_accounts_csv.withColumn('age', F.when(F.col('initial_deposit') <= 20000, quinn.array_choice(F.array(ages_young)))
                           .otherwise(quinn.array_choice(F.array(ages_adult))))


An error was encountered:
No module named 'quinn'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'quinn'



In [5]:
AML_accounts_fg_meta = fs.create_feature_group(name="aml_10k_accounts_fg",
                                        version=1,
                                        primary_key=['ncodpers'],
                                        description="Account generated using AMLSim",
                                        time_travel_format=None,                                        
                                        statistics_config={"enabled": True, "histograms": True, "correlations": True})    
# The following command does not work with parameter: features=['acct_id','dsply_nm','type','acct_stat','acct_rptng_crncy','prior_sar_count','branch_id','open_dt','close_dt','initial_deposit','tx_behavior_id','bank_id','country','age']
# The object required is a list of Features

In [6]:
AML_accounts_fg = AML_accounts_fg_meta.save(AML_accounts_csv)

In [7]:
AML_transactions_csv = spark.read\
             .option("inferSchema", "true")\
             .option("header", "true")\
             .format("csv")\
             .load("hdfs:///Projects/{}/Resources/transactions.csv".format(hdfs.project_name()))

AML_transcations_fg_meta = fs.create_feature_group(name="aml_10k_transactions_fg",
                                        version=1,
                                        primary_key=['ncodpers'],
                                        description="Transactions generated using AMLSim",
                                        time_travel_format=None,                                        
                                        statistics_config={"enabled": True, "histograms": True, "correlations": True})    
# The following command does not work with parameter: features=['acct_id','dsply_nm','type','acct_stat','acct_rptng_crncy','prior_sar_count','branch_id','open_dt','close_dt','initial_deposit','tx_behavior_id','bank_id','country','age']
# The object required is a list of Features


In [10]:
AML_transactions_fg = AML_transcations_fg_meta.save(AML_transactions_csv)